## Packages and Imports

In [1]:
#check for packages install if necessary
packages <- function(x){
  x <- as.character(match.call()[[2]])
  if (!require(x,character.only=TRUE)){
    install.packages(pkgs=x,repos="http://cran.r-project.org")
    require(x,character.only=TRUE)
  }
}

In [2]:
library(DescTools)
library(dplyr)
library(ggplot2)
library(readr)
library(readxl)
library(reshape2)
library(DT)



Attaching package: 'dplyr'

The following objects are masked from 'package:SparkR':

    arrange, between, collect, contains, count, cume_dist, dense_rank,
    desc, distinct, explain, filter, first, group_by, intersect, lag,
    last, lead, mutate, n, n_distinct, ntile, percent_rank, rename,
    row_number, sample_frac, select, sql, summarize

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



## Load in Raw Data from Ojbect Store

In [3]:
# The code was removed by DSX for sharing.

In [4]:
lcdf <- read.df(sqlContext, "swift://LendingClub.keystone/loan.small.csv", source = "com.databricks.spark.csv", header = "true")
#head(lcdf)
str(lcdf)


Formal class 'DataFrame' [package "SparkR"] with 2 slots
  ..@ env:<environment: 0x53be0e0> 
  ..@ sdf:Class 'jobj' <environment: 0x53b91a0> 


In [5]:
lcdf1 <- read.df(sqlContext, "swift://LendingClub.keystone/LoanStats_securev1_2016Q2.csv", source = "com.databricks.spark.csv", header = "true")
head(lcdf)

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
id
83830148
83032689
83758807
83983220
84112710


## Load Model from Github

In [8]:
load(url("https://github.com/dustinvanstee/random-public-files/raw/master/lrfit.Rdata"))

In [9]:
ls()
# head(lrfit)
lrfit$coefficients

[1] "emp_length_int" "h"              "json"           "lrfit"         
 [5] "newLoans"       "newLoans1"      "newdf"          "q"             
 [9] "req"            "sc"             "sqlContext"

(Intercept)                              annual_inc 
                           4.064349e+00                            5.055836e-07 
                         emp_length_int                       home_ownershipOWN 
                          -5.248582e-02                           -2.209914e+00 
                     home_ownershipRENT                               revol_bal 
                          -3.343001e+00                           -8.008158e-05 
                              loan_amnt                                open_acc 
                          -5.176806e-05                            1.596197e-02 
                  verification_status01            annual_inc:home_ownershipOWN 
                          -2.539360e-01                            2.092193e-05 
          annual_inc:home_ownershipRENT             home_ownershipOWN:revol_bal 
                           2.456082e-05                           -1.772855e-05 
           home_ownershipRENT:revol_bal                    annual_inc:revol_bal 
                           1.155185e-04                            4.142017e-10 
 annual_inc:home_ownershipOWN:revol_bal annual_inc:home_ownershipRENT:revol_bal 
                           1.758897e-10                           -1.041297e-09

In [12]:
# SAMPLES
#system("pwd 2>&1", intern = TRUE)
#n = c(2, 3, 5) 
#s = c("aa", "bb", "cc") 
#b = c(TRUE, FALSE, TRUE) 
#df = data.frame(n, s, b)    
#write.csv(df,'myCSVforGit.csv')
#system("pwd 2>&1", intern = TRUE)
#system("ls 2>&1", intern = TRUE)


[1] "/gpfs/fs01/user/s61e-f0fcc11d947a93-591ae43264f0/notebook/work"

## Test Prediction Code

In [8]:
record <- data.frame(
    annual_inc     = 50000, 
    emp_length_int = 5,
    home_ownership = "OWN",
    loan_amnt      = 10000, 
    open_acc       = 3, 
    revol_bal      = 6000, 
    loan_status01  = 1, 
    verification   = "Verified" , 
    verification_status01 = 0
)

In [9]:
prediction <- predict(lrfit, newdata = record, type = "response")
prediction

1 
0.8561598

# Grab data from Lending Club

In [1]:
library(curl)
h <- new_handle()
# handle_setopt(h, copypostfields = "moo=moomooo");
#curl -v -H "Authorization: ivTyQ0K1+9a3oaOSUDikuQiubVU="  "https://api.lendingclub.com/api/investor/v1/loans/listing?showAll=true"

handle_setheaders(h,
  "Authorization" = "ivTyQ0K1+9a3oaOSUDikuQiubVU="
)
req <- curl_fetch_memory("https://api.lendingclub.com/api/investor/v1/loans/listing?showAll=true", handle = h)



In [2]:
json <- rawToChar(req$content)

In [3]:
library(jsonlite)
newdf <- fromJSON(json)
newLoans <- newdf$loans

In [15]:
# Total number of new loans
nrow(newLoans)
# Trim Loans down to 10 to start
newLoans <- newLoans[1:10,]
library(plyr)
newLoans1<-rename(newLoans, c("annualInc"="annual_inc", 
            "empLength"="emp_length",
            "homeOwnership"="home_ownership",
            "revolBal"="revol_bal",
            "loanAmount"="loan_amnt",
            "openAcc"="open_acc",
            "verificationStatus"="verification_status"
           ))


newLoans2 <- addColumns(newLoans1)
#emp_length_int <- rep(10,10)
#newLoans1 <- cbind(newLoans1,emp_length_int)
names(newLoans1)

[1] 10

The following `from` values were not present in `x`: verificationStatus


[1] "id"                         "memberId"                  
  [3] "loan_amnt"                  "fundedAmount"              
  [5] "term"                       "intRate"                   
  [7] "expDefaultRate"             "serviceFeeRate"            
  [9] "installment"                "grade"                     
 [11] "subGrade"                   "emp_length"                
 [13] "home_ownership"             "annual_inc"                
 [15] "isIncV"                     "acceptD"                   
 [17] "expD"                       "listD"                     
 [19] "creditPullD"                "reviewStatusD"             
 [21] "reviewStatus"               "desc"                      
 [23] "purpose"                    "addrZip"                   
 [25] "addrState"                  "investorCount"             
 [27] "ilsExpD"                    "initialListStatus"         
 [29] "empTitle"                   "accNowDelinq"              
 [31] "accOpenPast24Mths"          "bcOpenToBuy"               
 [33] "percentBcGt75"              "bcUtil"                    
 [35] "dti"                        "delinq2Yrs"                
 [37] "delinqAmnt"                 "earliestCrLine"            
 [39] "ficoRangeLow"               "ficoRangeHigh"             
 [41] "inqLast6Mths"               "mthsSinceLastDelinq"       
 [43] "mthsSinceLastRecord"        "mthsSinceRecentInq"        
 [45] "mthsSinceRecentRevolDelinq" "mthsSinceRecentBc"         
 [47] "mortAcc"                    "open_acc"                  
 [49] "pubRec"                     "totalBalExMort"            
 [51] "revol_bal"                  "revolUtil"                 
 [53] "totalBcLimit"               "totalAcc"                  
 [55] "totalIlHighCreditLimit"     "numRevAccts"               
 [57] "mthsSinceRecentBcDlq"       "pubRecBankruptcies"        
 [59] "numAcctsEver120Ppd"         "chargeoffWithin12Mths"     
 [61] "collections12MthsExMed"     "taxLiens"                  
 [63] "mthsSinceLastMajorDerog"    "numSats"                   
 [65] "numTlOpPast12m"             "moSinRcntTl"               
 [67] "totHiCredLim"               "totCurBal"                 
 [69] "avgCurBal"                  "numBcTl"                   
 [71] "numActvBcTl"                "numBcSats"                 
 [73] "pctTlNvrDlq"                "numTl90gDpd24m"            
 [75] "numTl30dpd"                 "numTl120dpd2m"             
 [77] "numIlTl"                    "moSinOldIlAcct"            
 [79] "numActvRevTl"               "moSinOldRevTlOp"           
 [81] "moSinRcntRevTlOp"           "totalRevHiLim"             
 [83] "numRevTlBalGt0"             "numOpRevTl"                
 [85] "totCollAmt"                 "applicationType"           
 [87] "annualIncJoint"             "dtiJoint"                  
 [89] "isIncVJoint"                "openAcc6m"                 
 [91] "openIl6m"                   "openIl12m"                 
 [93] "openIl24m"                  "mthsSinceRcntIl"           
 [95] "totalBalIl"                 "iLUtil"                    
 [97] "openRv12m"                  "openRv24m"                 
 [99] "maxBalBc"                   "allUtil"                   
[101] "inqFi"                      "totalCuTl"                 
[103] "inqLast12m"

In [14]:
newLoans2 <- addColumns(newLoans1)

In [10]:
prediction <- predict(lrfit, newdata = newLoans1, type = "response")




ERROR: Error in eval(expr, envir, enclos): object 'verification_status01' not found
